# Find a suiting epsilon

The goal of this notebook is to derive a suitable magnitude of the attack for each feature.

In [1]:
#import uproot
import uproot4 as uproot
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sns
import mplhep as hep

In [2]:
import awkward1 as ak

In [3]:
uproot.__version__

'0.1.2'

## Checking files content

Focusing on pred_ntuple_merged_342

In [5]:
inputs_root = "/eos/cms/store/group/phys_btag/ParticleTransformer/merged/ntuple_merged_342.root:deepntuplizer/tree"

In [6]:
file_ = uproot.open(inputs_root)
columns_ = file_.keys()

In [7]:
columns_

['n_sv',
 'nsv',
 'sv_pt',
 'sv_eta',
 'sv_phi',
 'sv_e',
 'sv_etarel',
 'sv_phirel',
 'sv_deltaR',
 'sv_mass',
 'sv_ntracks',
 'sv_chi2',
 'sv_ndf',
 'sv_normchi2',
 'sv_dxy',
 'sv_dxyerr',
 'sv_dxysig',
 'sv_d3d',
 'sv_d3derr',
 'sv_d3dsig',
 'sv_costhetasvpv',
 'sv_enratio',
 'n_gtracks',
 'nGtracks',
 'gtrack_pt',
 'gtrack_eta',
 'gtrack_phi',
 'gtrack_mass',
 'gtrack_dz',
 'gtrack_dxy',
 'gtrack_3D_ip',
 'gtrack_3D_sip',
 'gtrack_2D_ip',
 'gtrack_2D_sip',
 'gtrack_dR',
 'gtrack_dist_neigh',
 'gtrack_3D_TrackProbability',
 'gtrack_2D_TrackProbability',
 'gtrack_chi2reduced',
 'gtrack_nPixelHits',
 'gtrack_nHits',
 'gtrack_jetAxisDistance',
 'gtrack_jetAxisDlength',
 'gtrack_PCAtrackFromPV',
 'gtrack_dotProdTrack',
 'gtrack_dotProdTrack2D',
 'npv',
 'rho',
 'ntrueInt',
 'event_no',
 'jet_no',
 'gen_pt',
 'Delta_gen_pt',
 'isB',
 'isGBB',
 'isBB',
 'isLeptonicB',
 'isLeptonicB_C',
 'isC',
 'isGCC',
 'isCC',
 'isUD',
 'isS',
 'isG',
 'isPU',
 'isUndefined',
 'genDecay',
 'jet_hflav',


In [8]:
global_branches = ['jet_pt',
                   'jet_eta',
                   'nCpfcand',
                   'nNpfcand',
                   'nsv',
                   'npv',
                   'TagVarCSV_trackSumJetEtRatio',
                   'TagVarCSV_trackSumJetDeltaR',
                   'TagVarCSV_vertexCategory',
                   'TagVarCSV_trackSip2dValAboveCharm',
                   'TagVarCSV_trackSip2dSigAboveCharm',
                   'TagVarCSV_trackSip3dValAboveCharm',
                   'TagVarCSV_trackSip3dSigAboveCharm',
                   'TagVarCSV_jetNSelectedTracks',
                   'TagVarCSV_jetNTracksEtaRel']
cpf_branches = ['Cpfcan_BtagPf_trackEtaRel',
                'Cpfcan_BtagPf_trackPtRel',
                'Cpfcan_BtagPf_trackPPar',
                'Cpfcan_BtagPf_trackDeltaR',
                'Cpfcan_BtagPf_trackPParRatio',
                'Cpfcan_BtagPf_trackSip2dVal',
                'Cpfcan_BtagPf_trackSip2dSig',
                'Cpfcan_BtagPf_trackSip3dVal',
                'Cpfcan_BtagPf_trackSip3dSig',
                'Cpfcan_BtagPf_trackJetDistVal',
                'Cpfcan_ptrel',
                'Cpfcan_drminsv',
                'Cpfcan_distminsv',
                'Cpfcan_VTX_ass',
                'Cpfcan_puppiw',
                'Cpfcan_chi2',
                'Cpfcan_quality']
npf_branches = ['Npfcan_ptrel',
                'Npfcan_etarel',
                'Npfcan_phirel', 
                'Npfcan_deltaR',
                'Npfcan_isGamma',
                'Npfcan_HadFrac',
                'Npfcan_drminsv',
                'Npfcan_puppiw']
vtx_branches = ['sv_pt',
                'sv_deltaR',
                'sv_mass',
                'sv_etarel',
                'sv_phirel',
                'sv_ntracks',
                'sv_chi2',
                'sv_normchi2',
                'sv_dxy',
                'sv_dxysig',
                'sv_d3d',
                'sv_d3dsig',
                'sv_costhetasvpv',
                'sv_enratio']
cpf_pts_branches = ['Cpfcan_pt',
                    'Cpfcan_eta',
                    'Cpfcan_phi',
                    'Cpfcan_e']
npf_pts_branches = ['Npfcan_pt',
                    'Npfcan_eta',
                    'Npfcan_phi',
                    'Npfcan_e']
vtx_pts_branches = ['sv_pt',
                    'sv_eta',
                    'sv_phi',
                    'sv_e']

In [9]:
df_glob = file_.arrays(global_branches, library="ak")

In [10]:
df_cpf = file_.arrays(cpf_branches, library="ak")

In [11]:
df_npf = file_.arrays(npf_branches, library="ak")

In [12]:
df_vtx = file_.arrays(vtx_branches, library="ak")

In [13]:
df_cpf_pts = file_.arrays(cpf_pts_branches, library="ak")

In [14]:
df_npf_pts = file_.arrays(npf_pts_branches, library="ak")

In [15]:
df_vtx_pts = file_.arrays(vtx_pts_branches, library="ak")

In [16]:
df_glob

<Array [{jet_pt: 24.3, ... ] type='400000 * {"jet_pt": float32, "jet_eta": float...'>

In [17]:
df_cpf

<Array [{Cpfcan_BtagPf_trackEtaRel: [, ... ] type='400000 * {"Cpfcan_BtagPf_trac...'>

In [18]:
df_npf

<Array [{Npfcan_ptrel: [-0.918, ... 0.969]}] type='400000 * {"Npfcan_ptrel": var...'>

In [19]:
df_vtx

<Array [{sv_pt: [], ... sv_enratio: []}] type='400000 * {"sv_pt": var * float32,...'>

In [20]:
df_cpf_pts

<Array [{Cpfcan_pt: [1.39, 12.7, ... 3.57]}] type='400000 * {"Cpfcan_pt": var * ...'>

In [21]:
df_npf_pts

<Array [{Npfcan_pt: [1.99, 1.17, ... 2.83]}] type='400000 * {"Npfcan_pt": var * ...'>

In [22]:
df_vtx_pts

<Array [{sv_pt: [], sv_eta: [, ... sv_e: []}] type='400000 * {"sv_pt": var * flo...'>

In [23]:
df_cpf_clip = ak.pad_none(df_cpf,25,clip=True)

In [24]:
df_npf_clip = ak.pad_none(df_npf,25,clip=True)

In [25]:
df_vtx_clip = ak.pad_none(df_vtx,5,clip=True)

In [26]:
df_cpf_pts_clip = ak.pad_none(df_cpf_pts,25,clip=True)

In [27]:
df_npf_pts_clip = ak.pad_none(df_npf_pts,25,clip=True)

In [28]:
df_vtx_pts_clip = ak.pad_none(df_vtx_pts,5,clip=True)

In [29]:
df_vtx_clip.type

400000 * {"sv_pt": 5 * ?float32, "sv_deltaR": 5 * ?float32, "sv_mass": 5 * ?float32, "sv_etarel": 5 * ?float32, "sv_phirel": 5 * ?float32, "sv_ntracks": 5 * ?float32, "sv_chi2": 5 * ?float32, "sv_normchi2": 5 * ?float32, "sv_dxy": 5 * ?float32, "sv_dxysig": 5 * ?float32, "sv_d3d": 5 * ?float32, "sv_d3dsig": 5 * ?float32, "sv_costhetasvpv": 5 * ?float32, "sv_enratio": 5 * ?float32}

In [30]:
df_vtx_pts_clip.type

400000 * {"sv_pt": 5 * ?float32, "sv_eta": 5 * ?float32, "sv_phi": 5 * ?float32, "sv_e": 5 * ?float32}

In [31]:
def quantile_min_max(feature,group='glob',candidate=None):
    if group=='glob':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_glob[feature])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='cpf':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_cpf_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='npf':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_npf_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='vtx':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_vtx_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='cpf_pts':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_cpf_pts_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='npf_pts':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_npf_pts_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]
    elif group=='vtx_pts':
        print(feature,group,candidate)
        array_np = ak.to_numpy(df_vtx_pts_clip[feature][:,candidate])
        array_np = np.where(array_np == -999, 0, array_np)
        array_np = np.where(array_np ==   -1, 0, array_np)
        return [np.quantile(array_np,0.01),np.quantile(array_np,0.99)]

In [32]:
print(quantile_min_max('jet_pt','glob'))

jet_pt glob None
[15.444969825744629, 1535.685830078128]


In [33]:
print(quantile_min_max('Cpfcan_BtagPf_trackEtaRel','cpf',1))

Cpfcan_BtagPf_trackEtaRel cpf 1
[1.715033929347992, 128.14074356079138]


In [34]:
global_epsilons = np.zeros(len(global_branches))

In [35]:
global_ranges = np.zeros((len(global_branches),2))

In [36]:
for (i,key) in enumerate(global_branches):
    range_inputs = quantile_min_max(key,'glob')
    scale_epsilon = (range_inputs[1] - range_inputs[0])/2
    global_epsilons[i] = scale_epsilon
    global_ranges[i] = range_inputs
    print(range_inputs, scale_epsilon)

jet_pt glob None
[15.444969825744629, 1535.685830078128] 760.1204301261916
jet_eta glob None
[-2.391017284393311, 2.3943071389198307] 2.392662211656571
nCpfcand glob None
[1.0, 36.0] 17.5
nNpfcand glob None
[0.0, 20.0] 10.0
nsv glob None
[0.0, 4.0] 2.0
npv glob None
[9.0, 48.0] 19.5
TagVarCSV_trackSumJetEtRatio glob None
[0.0, 0.9856463485956193] 0.49282317429780964
TagVarCSV_trackSumJetDeltaR glob None
[0.0, 2.6554229521751425] 1.3277114760875712
TagVarCSV_vertexCategory glob None
[0.0, 2.0] 1.0
TagVarCSV_trackSip2dValAboveCharm glob None
[-0.014902344904839993, 0.036601562052965164] 0.02575195347890258
TagVarCSV_trackSip2dSigAboveCharm glob None
[-2.80285924911499, 12.758635606765749] 7.780747427940369
TagVarCSV_trackSip3dValAboveCharm glob None
[-0.029931544382125138, 0.06389331802725803] 0.04691243120469159
TagVarCSV_trackSip3dSigAboveCharm glob None
[-5.315430493354796, 18.9747463989258] 12.145088446140297
TagVarCSV_jetNSelectedTracks glob None
[0.0, 17.0] 8.5
TagVarCSV_jetNTracks

In [37]:
global_ranges

array([[ 1.54449698e+01,  1.53568583e+03],
       [-2.39101728e+00,  2.39430714e+00],
       [ 1.00000000e+00,  3.60000000e+01],
       [ 0.00000000e+00,  2.00000000e+01],
       [ 0.00000000e+00,  4.00000000e+00],
       [ 9.00000000e+00,  4.80000000e+01],
       [ 0.00000000e+00,  9.85646349e-01],
       [ 0.00000000e+00,  2.65542295e+00],
       [ 0.00000000e+00,  2.00000000e+00],
       [-1.49023449e-02,  3.66015621e-02],
       [-2.80285925e+00,  1.27586356e+01],
       [-2.99315444e-02,  6.38933180e-02],
       [-5.31543049e+00,  1.89747464e+01],
       [ 0.00000000e+00,  1.70000000e+01],
       [ 0.00000000e+00,  9.00000000e+00]])

In [38]:
cpf_epsilons = np.zeros((len(cpf_branches),25))

In [39]:
cpf_ranges = np.zeros((len(cpf_branches),25, 2))

In [40]:
for (i,key) in enumerate(cpf_branches):
    for cand in range(25):
        range_inputs = quantile_min_max(key,'cpf',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        cpf_epsilons[i,cand] = scale_epsilon
        cpf_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

Cpfcan_BtagPf_trackEtaRel cpf 0
[1.6402074801921847, 6.027635936737061] 2.1937142282724382
Cpfcan_BtagPf_trackEtaRel cpf 1
[1.6706652629375458, 6.259780974388123] 2.2945578557252886
Cpfcan_BtagPf_trackEtaRel cpf 2
[1.6765463149547577, 6.338044652938844] 2.3307491689920434
Cpfcan_BtagPf_trackEtaRel cpf 3
[1.678204838037491, 6.380094428062439] 2.350944795012474
Cpfcan_BtagPf_trackEtaRel cpf 4
[1.677532956600189, 6.3942133235931395] 2.358340183496475
Cpfcan_BtagPf_trackEtaRel cpf 5
[0.0, 6.211091284751893] 3.1055456423759464
Cpfcan_BtagPf_trackEtaRel cpf 6
[0.0, 6.171953864097597] 3.0859769320487986
Cpfcan_BtagPf_trackEtaRel cpf 7
[0.0, 6.173353486061096] 3.086676743030548
Cpfcan_BtagPf_trackEtaRel cpf 8
[0.0, 6.1727183675766] 3.0863591837883
Cpfcan_BtagPf_trackEtaRel cpf 9
[0.0, 6.1843814039230365] 3.0921907019615182
Cpfcan_BtagPf_trackEtaRel cpf 10
[0.0, 6.210726642608644] 3.105363321304322
Cpfcan_BtagPf_trackEtaRel cpf 11
[0.0, 6.214005861282349] 3.1070029306411744
Cpfcan_BtagPf_trackE

In [41]:
npf_epsilons = np.zeros((len(npf_branches),25))

In [42]:
npf_ranges = np.zeros((len(npf_branches),25, 2))

In [43]:
for (i,key) in enumerate(npf_branches):
    for cand in range(25):
        range_inputs = quantile_min_max(key,'npf',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        npf_epsilons[i,cand] = scale_epsilon
        npf_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

Npfcan_ptrel npf 0
[-0.996482680439949, 4.1626909543191853e-39] 0.4982413402199745
Npfcan_ptrel npf 1
[-0.9978396904468537, 4.1626909543191853e-39] 0.49891984522342686
Npfcan_ptrel npf 2
[-0.9985075026750565, 4.165400841341424e-39] 0.49925375133752825
Npfcan_ptrel npf 3
[-0.9986718302965164, 2.3901500891778286e-39] 0.4993359151482582
Npfcan_ptrel npf 4
[-0.9987648129463196, 2.6908576693640664e-40] 0.4993824064731598
Npfcan_ptrel npf 5
[-0.9989252686500549, 2.597489432945797e-40] 0.49946263432502747
Npfcan_ptrel npf 6
[-0.9989713460206986, 2.597489432945797e-40] 0.4994856730103493
Npfcan_ptrel npf 7
[-0.9989966160058975, 2.597489432945797e-40] 0.49949830800294875
Npfcan_ptrel npf 8
[-0.9990305310487747, 2.597489432945797e-40] 0.49951526552438735
Npfcan_ptrel npf 9
[-0.999043345451355, 2.597489432945797e-40] 0.4995216727256775
Npfcan_ptrel npf 10
[-0.9990576505661011, 2.597489432945797e-40] 0.49952882528305054
Npfcan_ptrel npf 11
[-0.999071598649025, 2.597489432945797e-40] 0.499535799324

In [44]:
vtx_epsilons = np.zeros((len(vtx_branches),5))

In [45]:
vtx_ranges = np.zeros((len(vtx_branches),5, 2))

In [47]:
for (i,key) in enumerate(vtx_branches):
    for cand in range(5):
        range_inputs = quantile_min_max(key,'vtx',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        vtx_epsilons[i,cand] = scale_epsilon
        vtx_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

sv_pt vtx 0
[0.0, 270.0095565795899] 135.00477828979496
sv_pt vtx 1
[0.0, 281.2295611572271] 140.61478057861356
sv_pt vtx 2
[0.0, 286.7119869995122] 143.3559934997561
sv_pt vtx 3
[0.0, 287.9476156616214] 143.9738078308107
sv_pt vtx 4
[0.0, 288.7673446655274] 144.3836723327637
sv_deltaR vtx 0
[-0.4964105808734894, 1.0] 0.7482052904367447
sv_deltaR vtx 1
[-0.49653978377580643, 1.0] 0.7482698918879032
sv_deltaR vtx 2
[-0.49655191361904144, 1.0] 0.7482759568095207
sv_deltaR vtx 3
[-0.4965553012490273, 1.0] 0.7482776506245137
sv_deltaR vtx 4
[-0.49655975311994555, 1.0] 0.7482798765599727
sv_mass vtx 0
[0.0, 9.334539136886615] 4.667269568443308
sv_mass vtx 1
[0.0, 10.931052799224876] 5.465526399612438
sv_mass vtx 2
[0.0, 11.528788757324241] 5.764394378662121
sv_mass vtx 3
[0.0, 11.732430810928365] 5.866215405464183
sv_mass vtx 4
[0.0, 11.792854328155522] 5.896427164077761
sv_etarel vtx 0
[-0.49928516149520874, 1.0] 0.7496425807476044
sv_etarel vtx 1
[-0.499312165081501, 1.0] 0.74965608254075

In [48]:
cpf_pts_epsilons = np.zeros((len(cpf_pts_branches),25))

In [49]:
cpf_pts_ranges = np.zeros((len(cpf_pts_branches),25, 2))

In [50]:
for (i,key) in enumerate(cpf_pts_branches):
    for cand in range(25):
        range_inputs = quantile_min_max(key,'cpf_pts',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        cpf_pts_epsilons[i,cand] = scale_epsilon
        cpf_pts_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

Cpfcan_pt cpf_pts 0
[0.9580078125, 73.0625] 36.05224609375
Cpfcan_pt cpf_pts 1
[0.96044921875, 101.93812500000058] 50.48883789062529
Cpfcan_pt cpf_pts 2
[0.9609375, 114.68812500000058] 56.86359375000029
Cpfcan_pt cpf_pts 3
[0.9609375, 119.875] 59.45703125
Cpfcan_pt cpf_pts 4
[0.96044921875, 119.8125] 59.426025390625
Cpfcan_pt cpf_pts 5
[0.0, 108.5625] 54.28125
Cpfcan_pt cpf_pts 6
[0.0, 104.125] 52.0625
Cpfcan_pt cpf_pts 7
[0.0, 105.9375] 52.96875
Cpfcan_pt cpf_pts 8
[0.0, 105.62562500000058] 52.81281250000029
Cpfcan_pt cpf_pts 9
[0.0, 107.0] 53.5
Cpfcan_pt cpf_pts 10
[0.0, 109.43812500000058] 54.71906250000029
Cpfcan_pt cpf_pts 11
[0.0, 109.00062500000058] 54.50031250000029
Cpfcan_pt cpf_pts 12
[0.0, 109.5] 54.75
Cpfcan_pt cpf_pts 13
[0.0, 111.87562500000058] 55.93781250000029
Cpfcan_pt cpf_pts 14
[0.0, 112.625] 56.3125
Cpfcan_pt cpf_pts 15
[0.0, 113.375] 56.6875
Cpfcan_pt cpf_pts 16
[0.0, 113.5] 56.75
Cpfcan_pt cpf_pts 17
[0.0, 114.625] 57.3125
Cpfcan_pt cpf_pts 18
[0.0, 114.9375] 57.

In [51]:
npf_pts_epsilons = np.zeros((len(npf_pts_branches),25))

In [52]:
npf_pts_ranges = np.zeros((len(npf_pts_branches),25, 2))

In [53]:
for (i,key) in enumerate(npf_pts_branches):
    for cand in range(25):
        range_inputs = quantile_min_max(key,'npf_pts',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        npf_pts_epsilons[i,cand] = scale_epsilon
        npf_pts_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

Npfcan_pt npf_pts 0
[0.0, 627.5050000000047] 313.7525000000023
Npfcan_pt npf_pts 1
[0.0, 347.75] 173.875
Npfcan_pt npf_pts 2
[0.0, 124.31312500000058] 62.15656250000029
Npfcan_pt npf_pts 3
[0.0, 75.625] 37.8125
Npfcan_pt npf_pts 4
[0.0, 41.75031250000029] 20.875156250000146
Npfcan_pt npf_pts 5
[0.0, 56.18781250000029] 28.093906250000146
Npfcan_pt npf_pts 6
[0.0, 46.75031250000029] 23.375156250000146
Npfcan_pt npf_pts 7
[0.0, 41.03125] 20.515625
Npfcan_pt npf_pts 8
[0.0, 36.25031250000029] 18.125156250000146
Npfcan_pt npf_pts 9
[0.0, 33.8125] 16.90625
Npfcan_pt npf_pts 10
[0.0, 31.765625] 15.8828125
Npfcan_pt npf_pts 11
[0.0, 30.0625] 15.03125
Npfcan_pt npf_pts 12
[0.0, 28.890625] 14.4453125
Npfcan_pt npf_pts 13
[0.0, 27.765625] 13.8828125
Npfcan_pt npf_pts 14
[0.0, 26.9375] 13.46875
Npfcan_pt npf_pts 15
[0.0, 26.609375] 13.3046875
Npfcan_pt npf_pts 16
[0.0, 26.203281250000146] 13.101640625000073
Npfcan_pt npf_pts 17
[0.0, 26.046875] 13.0234375
Npfcan_pt npf_pts 18
[0.0, 25.828125] 12.9

In [54]:
vtx_pts_epsilons = np.zeros((len(vtx_pts_branches),5))

In [55]:
vtx_pts_ranges = np.zeros((len(vtx_pts_branches),5, 2))

In [56]:
for (i,key) in enumerate(vtx_pts_branches):
    for cand in range(5):
        range_inputs = quantile_min_max(key,'vtx_pts',cand)
        scale_epsilon = (range_inputs[1] - range_inputs[0])/2
        vtx_epsilons[i,cand] = scale_epsilon
        vtx_ranges[i,cand] = range_inputs
        print(range_inputs, scale_epsilon)

sv_pt vtx_pts 0
[0.0, 280.3638479614259] 140.18192398071295
sv_pt vtx_pts 1
[0.0, 291.59263946533275] 145.79631973266638
sv_pt vtx_pts 2
[0.0, 297.007989807129] 148.5039949035645
sv_pt vtx_pts 3
[0.0, 298.06402343750057] 149.03201171875028
sv_pt vtx_pts 4
[0.0, 298.93584014892633] 149.46792007446317
sv_eta vtx_pts 0
[-2.2064412951469423, 24.02778165817267] 13.117111476659806
sv_eta vtx_pts 1
[-2.207332701683044, 24.034761562347654] 13.121047132015349
sv_eta vtx_pts 2
[-2.207797219753265, 24.079509525299123] 13.143653372526193
sv_eta vtx_pts 3
[-2.207797219753265, 24.079509525299123] 13.143653372526193
sv_eta vtx_pts 4
[-2.2080804085731507, 24.079509525299123] 13.143794966936136
sv_phi vtx_pts 0
[-3.0113272953033445, 24.02778165817267] 13.519554476738008
sv_phi vtx_pts 1
[-3.0115152168273926, 24.034761562347654] 13.523138389587523
sv_phi vtx_pts 2
[-3.0111592197418213, 24.079509525299123] 13.545334372520472
sv_phi vtx_pts 3
[-3.0116208887100218, 24.079509525299123] 13.545565207004572
sv

In [57]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/global_epsilons.npy',global_epsilons)

In [58]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/global_ranges.npy',global_ranges)

In [59]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/cpf_epsilons.npy',cpf_epsilons)

In [60]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/cpf_ranges.npy',cpf_ranges)

In [61]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/npf_epsilons.npy',npf_epsilons)

In [62]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/npf_ranges.npy',npf_ranges)

In [63]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/vtx_epsilons.npy',vtx_epsilons)

In [64]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/vtx_ranges.npy',vtx_ranges)

In [65]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/cpf_pts_epsilons.npy',cpf_pts_epsilons)

In [66]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/cpf_pts_ranges.npy',cpf_pts_ranges)

In [67]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/npf_pts_epsilons.npy',npf_pts_epsilons)

In [68]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/npf_pts_ranges.npy',npf_pts_ranges)

In [69]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/vtx_pts_epsilons.npy',vtx_pts_epsilons)

In [70]:
np.save('/eos/user/a/anstein/public/DeepJet/Train_ParT/auxiliary/vtx_pts_ranges.npy',vtx_pts_ranges)